In [ ]:
# TODO: requirements.txt
# pip install binreader
# pip install matplotlib
# pip install numpy
# pip install nbconvert

In [ ]:
from typing import Callable

# Import
from pupil_info import *
import matplotlib.pyplot as plt
import numpy as np
from local_features import *
from recording import Measure, MidiNote, Recording
from typing import Callable
from matplotlib.axes import Axes
from matplotlib.figure import Figure


INFO: list[PupilInfo] = get_all_pupil_info()

In [ ]:
# GET THE AVERAGE ANSWER FOR EVERY QUESTION FOR EVERY SONG

for question in range(NUM_QUESTIONS):
    print(f"Question {question}")
    for song in range(NUM_SONGS):
        res = [
            INFO[pupil].sessions[session].performances[performance].responses[question]
            for pupil in range(NUM_PUPILS)
            for session in range(NUM_SESSIONS)
            for performance in range(NUM_PERFORMANCES)
            if INFO[pupil].sessions[session].performances[performance].song == song
        ]
        res = list(filter(lambda x: x != 0, res))
        print(f"{SONG_NAMES[song]}: {np.average(res)}")
    print()

In [ ]:
# GET THE AVERAGE ANSWER PER QUESTION OVER TIME

for question in range(NUM_QUESTIONS):
    print(f"Question {question}")
    for session in range(NUM_SESSIONS):
        res = [INFO[pupil].sessions[session].performances[performance].self_report[question]
               for pupil in range(NUM_PUPILS) for performance in range(NUM_PERFORMANCES)]
        res = list(filter(lambda x: x != 0, res))
        print(f"Session {session}: {np.average(res)}")

In [ ]:
from dataclasses import replace


fig, ax = plt.subplots()

colors = [
    'b', 'g', 'r', 'c', 'm'
]

def make_notes_absolute(measures: list[Measure]) -> list[Measure]:
    return [Measure([replace(note, time=(note.time + i)) for note in measures[i].notes]) for i in range(len(measures))]

for session in range(2):
    for pupil in range(NUM_PUPILS):
        if (pupil, session) == (2, 1): continue

        p = 0

        recording: Recording = Recording(f"recordings/{session + 1}/{pupil + 1}/{p}.notes")

        song = INFO[pupil].sessions[session].performances[p].song
        pitch_classes = SONG_PITCH_CLASSES[SONG_NAMES[song]]

        # recording: Recording = all_pupil_info[pupil].sessions[session].performances[p].recording



        metrics = [note_share(four, pitch_classes[2]) for four in fours]

        ax.plot(metrics, color=colors[song])

plt.show()

In [ ]:

# Are some songs simply more fun than others?

# Evaluate this specified question
question_index: int = 3

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

# Keep track of all scores
all_scores: list[list[int]] = []

song_index: int
for song_index in range(NUM_SONGS):
    # Get all scores
    scores: list[int] = [performance.responses[question_index] for performance in all_performances if performance.song == song_index]

    # (TODO: remove) Filter all not-yet-filled-in answers
    scores = [score for score in scores if score > 0]

    # Append to plotting variable
    all_scores.append(scores)

    # print(SONG_NAMES[song], ": ", np.average(scores), sep='')

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_scores, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)
ax.set_ylim([1, 5.2])
_ = ax.set_xticks([y + 1 for y in range(NUM_SONGS)], labels=list(SONG_IDS.keys()))


In [ ]:
# Are pupils self-reporting feeling more musical over time?
# Are pupils self-reporting feeling more response (from themselves; from the algorithm) over time?
# Are pupils self-reporting feeling more enjoyment over time?

# Evaluate this specified question
question_index: int = 3

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get all performances of the session
    performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get the question scores
    scores: list[int] = [
        performance.responses[question_index]
        for performance in performances
    ]

    # (TODO: remove) Filter all not-yet-filled-in answers
    scores = [score for score in scores if score > 0]

    print(session_index, ": ", np.average(scores), sep='')


In [ ]:
# Are some algorithms simply more fun than others?

# Evaluate this specified question and algorithm
question_index: int = 3

# Get all performances
all_performances: list[PerformanceInfo] = [
    INFO[pupil_index].sessions[session_index].performances[performance_index]
    for pupil_index in range(NUM_PUPILS)
    for session_index in range(NUM_SESSIONS)
    for performance_index in range(NUM_PERFORMANCES)
]

algorithm_index: int
for algorithm_index in range(NUM_ALGORITHMS):
    # Get scores of performances of algorithm
    scores: list[int] = [
        performance.responses[question_index]
        for performance in all_performances
        if performance.algorithm == algorithm_index
    ]

    # (TODO: remove) Filter all not-yet-filled-in answers
    scores = [score for score in scores if score > 0]

    print(ALGORITHM_NAMES[algorithm_index], ": ", np.average(scores), sep='')


In [ ]:
# For a specific algorithm, for a specific metric, how does it (and its distribution) evolve over the sessions for a specific algorithm?

# Evaluate this specific metric
metric: Callable[[list[Measure]], float]
metric = melodic_arc_height_avg

# Evaluate this specific algorithm
algorithm_index: int = 0

all_metrics: list[list[float]] = []

session_index: int
for session_index in range(NUM_SESSIONS):
    # Get performances
    performances: list[PerformanceInfo] = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Get metrics
    metrics: list[float] = [
        metric(four)
        for performance in performances
        for four in performance.recording.fours
    ]

    all_metrics.append(metrics)

fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_metrics, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)

plt.show()

In [ ]:
# (out of personal interest) Is the fifth really more popular in solos of Summertime?

from functools import partial

all_metrics: list[list[float]] = []

for song_index in range(NUM_SONGS):
    fifth = SONG_PITCH_CLASSES[SONG_NAMES[song_index]][2]

    # Get performances
    performances = [
        INFO[pupil_index].sessions[session_index].performances[performance_index]
        for pupil_index in range(NUM_PUPILS)
        for session_index in range(NUM_SESSIONS)
        for performance_index in range(NUM_PERFORMANCES)
    ]

    # Filter by song
    performances = [
        performance
        for performance in performances
        if performance.song == song_index
    ]

    # Get metrics
    metrics: list[float] = [
        note_share(four, pitch_class=fifth)
        for performance in performances
        for four in performance.recording.fours
    ]

    all_metrics.append(metrics)


fig: Figure
ax: Axes
fig, ax = plt.subplots()
ax.violinplot(all_metrics, showmeans=True)
ax.set_title('Violin plot')

ax.yaxis.grid(True)

plt.show()

In [ ]:
# For a specific question, how does its self-reporting score relate to the expert evaluation?



In [ ]:
# Piano roll for a specified performance
from recording import OutputName

performance: PerformanceInfo = INFO[2].sessions[0].performances[2]
print(SONG_NAMES[performance.song], "with", ALGORITHM_NAMES[performance.algorithm])
recording = performance.recording
midi_notes: list[MidiNote] = all_notes(recording.measures)


fig: Figure
ax: Axes
fig, ax = plt.subplots(figsize=(20, 5))

# Plot each note with the corresponding color
for note in midi_notes:
    color = "red" if note.output_name == OutputName.LOOPBACK else "blue"
    ax.broken_barh([(note.time, note.length)], (note.note - 0.4, 0.8), facecolors=color)

# Formatting
ax.invert_yaxis()
ax.set_xticks(np.arange(0, 64, 4))
ax.set_yticks([])

ax.grid(True)

# Add legend
legend_patches = [
    plt.Line2D([0], [0], color="red", lw=4, label="Loopback"),
    plt.Line2D([0], [0], color="blue", lw=4, label="Algorithm")
]
ax.legend(handles=legend_patches, loc="upper right")

plt.show()